In [81]:
import pandas as pd
import matplotlib.pyplot as plt
import re


In [82]:
datos = pd.read_csv('/content/conjunto_de_datos_gastoshogar_enigh_2018_ns.csv')

datos.sample(n=6)

/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (14,23) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,folioviv,foliohog,clave,tipo_gasto,mes_dia,forma_pag1,forma_pag2,forma_pag3,lugar_comp,orga_inst,frecuencia,fecha_adqu,fecha_pago,cantidad,gasto,pago_mp,costo,inmujer,inst_1,inst_2,num_meses,num_pagos,ultim_pago,gasto_tri,gasto_nm,gas_nm_tri,imujer_tri
659552,1161394916,1,H078,G1,0,1,0,0,17,0,0,0,0,,200,200,,200,,,,,,195.65,,,195.65
286864,504788404,1,C002,G1,0,1,0,0,6,0,0,0,0,,48,,,,,,,,,139.35,,,
1040909,1908198303,1,F007,G1,0,1,0,0,0,0,0,0,0,,700,,,,,,,,,2032.25,,,
101538,260025907,1,H058,G1,0,1,0,0,2,0,0,0,0,,60,60,,,,,,,,58.69,,,
711210,1261621018,1,J030,G1,0,1,0,0,5,0,0,0,0,,180,0,,180,1,,,,,176.08,,,176.08
648979,1160709609,1,D007,G1,0,1,0,0,6,0,0,0,0,,108,,,108,,,,,,324,,,324


In [83]:
datos['folioviv'].nunique()

43536

In [84]:
datos['folioviv'].shape

(1048575,)

Recordemos que solo nos interesa el hogar principal , por lo que seleccionamos el hoga rprincial.

In [85]:
hogar_principal = datos['foliohog'] == 1

gasto_hogar_principal = datos[hogar_principal]

In [86]:
gasto_hogar_principal.shape

(1032509, 27)

pandas tiene un sistema de opciones que nos permite personalizar algunos aspectos de su comportamiento, siendo las opciones relacionadas con la visualización aquellas que el usuario probablemente ajuste. Veamos cómo establecer el valor de una opción específica.

Cambiamos el número de filas que se mostrarán usando `display.max_rows`.

In [87]:
pd.set_option('display.max_rows',None)

In [88]:
gasto_hogar_principal.isnull().sum()

folioviv      0
foliohog      0
clave         0
tipo_gasto    0
mes_dia       0
forma_pag1    0
forma_pag2    0
forma_pag3    0
lugar_comp    0
orga_inst     0
frecuencia    0
fecha_adqu    0
fecha_pago    0
cantidad      0
gasto         0
pago_mp       0
costo         0
inmujer       0
inst_1        0
inst_2        0
num_meses     0
num_pagos     0
ultim_pago    0
gasto_tri     0
gasto_nm      0
gas_nm_tri    0
imujer_tri    0
dtype: int64

In [89]:
gasto_hogar_principal.sample(n=12)

,folioviv,foliohog,clave,tipo_gasto,mes_dia,forma_pag1,forma_pag2,forma_pag3,lugar_comp,orga_inst,frecuencia,fecha_adqu,fecha_pago,cantidad,gasto,pago_mp,costo,inmujer,inst_1,inst_2,num_meses,num_pagos,ultim_pago,gasto_tri,gasto_nm,gas_nm_tri,imujer_tri
957136,1760006710,1,C006,G1,0,1,0,0,4,0,0,0,0,,15,,,,,,,,,45,,,
938745,1701714509,1,D001,G1,0,1,0,0,6,0,0,0,0,,108,,,54,,,,,,313.54,,,156.77
604260,1060630410,1,C014,G1,0,1,0,0,4,0,0,0,0,,70,,,,,,,,,203.22,,,
1039280,1907699005,1,D014,G1,0,8,0,0,6,0,0,0,0,,120,,,-1,,,,,,348.38,,,-1
101361,260025419,1,D015,G1,0,1,0,0,4,0,0,0,0,,50,,,50,,,,,,145.16,,,145.16
504185,900778506,1,J071,G1,0,1,0,0,0,0,0,0,0,,840,280,,280,1,,,,,821.73,,,273.91
331360,600272403,1,E027,G1,0,1,0,0,0,0,0,0,0,,299,,,150,,,,,,868.06,,,435.48
297178,505749406,1,H115,G1,0,1,0,0,5,0,0,0,0,,1500,1500,,1500,,,,,,1467.39,,,1467.39
614382,1103391806,1,C010,G1,0,1,0,0,6,0,0,0,0,,36,,,,,,,,,108,,,
1028118,1903891701,1,C003,G1,0,1,0,0,6,0,0,0,0,,12,,,,,,,,,36,,,


Notemos , que existen variables en las que no hay valores en especificos , esto se debe a que los hogars, tomemos por ejemplo la variable `inst_2`  , no  cuentan con apoyo de alguna institucion publica o privada que proporciona los producto y servicios para el cuidado de la salud , **¿Podriamos suponer que no estan asegurados?**

Tomemos como ejemplo otra variable como `inumer_tri` , donde hay filas donde no existe ningun valor , por lo que podemos preguntarnos que no hay gasto a la decena de levantamiento en productos  servicios obtenidos para las mujeres o niñas.

Procedamos a reemplazar los espacios usando regex , recordemos que queremos remplazar los espacios en blancos , por lo que usaremos  `\s\s*` , remplazandolas por el valor 0 .

In [90]:
df_updated = gasto_hogar_principal.replace(to_replace="\s\s*",value = '0',regex=True)

In [91]:
df_updated.sample(n=10)

,folioviv,foliohog,clave,tipo_gasto,mes_dia,forma_pag1,forma_pag2,forma_pag3,lugar_comp,orga_inst,frecuencia,fecha_adqu,fecha_pago,cantidad,gasto,pago_mp,costo,inmujer,inst_1,inst_2,num_meses,num_pagos,ultim_pago,gasto_tri,gasto_nm,gas_nm_tri,imujer_tri
219848,401160204,1,D011,G1,0,1,0,0,6,0,0,0,0,0,35,0,0,0,0,0,0,0,0,101.61,0,0,0
919209,1661316814,1,H056,G1,0,1,0,0,5,0,0,0,0,0,1080,360,0,0,0,0,0,0,0,1056.52,0,0,0
853667,1560481707,1,H104,G1,0,1,0,0,2,0,0,0,0,0,100,100,0,100,0,0,0,0,0,97.82,0,0,97.82
253243,501160902,1,D003,G1,0,1,0,0,6,0,0,0,0,0,24,0,0,24,0,0,0,0,0,69.67,0,0,69.67
520268,910833206,1,N016,G1,0,0,0,0,0,0,0,0,0,0,750,0,0,0,0,0,0,0,0,372.92,0,0,0
955078,1703378804,1,M017,G1,0,1,0,0,5,0,0,0,0,0,600,0,0,0,0,0,0,0,0,295.08,0,0,0
598117,1060386414,1,H072,G1,0,1,0,0,5,0,0,0,0,0,5000,0,0,5000,0,0,0,0,0,4891.3,0,0,4891.3
683221,1203754603,1,N013,G1,0,0,0,0,0,0,0,0,0,0,3500,3500,0,0,0,0,0,0,0,1721.31,0,0,0
3288,100211005,1,D010,G1,0,1,0,0,6,0,0,0,0,0,140,0,0,35,0,0,0,0,0,406.45,0,0,101.61
453517,805394403,1,D017,G1,0,1,0,0,6,0,0,0,0,0,300,0,0,300,0,0,0,0,0,870.96,0,0,870.96


In [92]:
df_updated.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1032509 entries, 0 to 1048574
Data columns (total 27 columns):
 #   Column      Non-Null Count    Dtype 
---  ------      --------------    ----- 
 0   folioviv    1032509 non-null  int64 
 1   foliohog    1032509 non-null  int64 
 2   clave       1032509 non-null  object
 3   tipo_gasto  1032509 non-null  object
 4   mes_dia     1032509 non-null  int64 
 5   forma_pag1  1032509 non-null  int64 
 6   forma_pag2  1032509 non-null  int64 
 7   forma_pag3  1032509 non-null  int64 
 8   lugar_comp  1032509 non-null  int64 
 9   orga_inst   1032509 non-null  int64 
 10  frecuencia  1032509 non-null  int64 
 11  fecha_adqu  1032509 non-null  int64 
 12  fecha_pago  1032509 non-null  int64 
 13  cantidad    1032509 non-null  object
 14  gasto       1032509 non-null  object
 15  pago_mp     1032509 non-null  object
 16  costo       1032509 non-null  object
 17  inmujer     1032509 non-null  object
 18  inst_1      1032509 non-null  object
 19  

Nos interesan las variables de `gasto` & gasto trimestral , aunque también seria itneresante promediar todooooos los tipos de gasto. 

In [93]:
df_updated['gasto'] = df_updated['gasto'].astype(float)
df_updated['gasto_tri']  = df_updated['gasto_tri'].astype(float)
df_updated['inmujer'] = df_updated['inmujer'].astype(float)
df_updated['gas_nm_tri']  = df_updated['gas_nm_tri'].astype(float)
df_updated['imujer_tri']  = df_updated['imujer_tri'].astype(float)
df_updated['costo'] = df_updated['costo'].astype(float)
df_updated['pago_mp']  = df_updated['pago_mp'].astype(float)

In [94]:
df_updated.dtypes

folioviv        int64
foliohog        int64
clave          object
tipo_gasto     object
mes_dia         int64
forma_pag1      int64
forma_pag2      int64
forma_pag3      int64
lugar_comp      int64
orga_inst       int64
frecuencia      int64
fecha_adqu      int64
fecha_pago      int64
cantidad       object
gasto         float64
pago_mp       float64
costo         float64
inmujer       float64
inst_1         object
inst_2         object
num_meses      object
num_pagos      object
ultim_pago     object
gasto_tri     float64
gasto_nm       object
gas_nm_tri    float64
imujer_tri    float64
dtype: object

Tomaremos en cuenta sol

In [95]:
gasto_tri_gast =  pd.melt(df_updated, id_vars=['folioviv'], value_vars=['gasto','pago_mp','inmujer'])
gasto_tri_gast.sample(n=6)

,folioviv,variable,value
48290,160150405,gasto,1755.0
1834216,1502297305,pago_mp,0.0
2600713,960011413,inmujer,0.0
1304237,503374804,pago_mp,600.0
691006,1260535421,gasto,10.0
1440828,761057617,pago_mp,0.0


In [96]:
gasto_tri_gast = gasto_tri_gast.groupby(['folioviv'])[['value']].mean()
gasto_tri_gast = gasto_tri_gast.rename(columns={"value": "Gasto Promedio*"})
gasto_tri_gast = gasto_tri_gast.reset_index()
gasto_tri_gast.head()

,folioviv,Gasto Promedio*
0,100013601,173.733333
1,100013602,236.492063
2,100013603,718.028986
3,100013604,422.000000
4,100013606,51.912281


In [98]:
gasto_promedio_t = gasto_tri_gast.drop(['folioviv'],axis='columns',inplace=True)

gasto_promedio_t

KeyError: ignored

In [78]:
gasto_trimestral =  pd.melt(df_updated, id_vars=['folioviv'], value_vars=['gasto_tri','imujer_tri'])
gasto_trimestral.sample(n=6)

,folioviv,variable,value
288508,505332709,gasto_tri,129.00
1314775,504644902,imujer_tri,0.00
636898,1160461505,gasto_tri,5806.45
233703,460155205,gasto_tri,522.58
484002,860376309,gasto_tri,156.77
883906,1604411916,gasto_tri,113.22


In [79]:
gasto_trimestral = gasto_trimestral.groupby(['folioviv'])[['value']].mean()
gasto_trimestral = gasto_trimestral.rename(columns={"value": "Gasto Trimestral Promedio"})
gasto_trimestral = gasto_trimestral.reset_index()
gasto_trimestral.head()

,folioviv,Gasto Trimestral Promedio
0,100013601,782.000000
1,100013602,269.924405
2,100013603,1268.942609
3,100013604,889.991250
4,100013606,170.228158


In [80]:
dataframe_final = pd.concat([gasto_tri_gast,gasto_trimestral], axis=1)

dataframe_final.head()

,folioviv,Gasto Promedio*,folioviv,Gasto Trimestral Promedio
0,100013601,173.733333,100013601,782.000000
1,100013602,236.492063,100013602,269.924405
2,100013603,718.028986,100013603,1268.942609
3,100013604,422.000000,100013604,889.991250
4,100013606,51.912281,100013606,170.228158
